In [1]:
import pandas as pd
import numpy as np

In [2]:
# IMPORT BATTING DATA AND ASSIGN HOF AS 0 FOR 'NO' AND 1 FOR 'YES' 
batting_df = pd.read_csv("Resources/batting_data.csv", index_col=0)
batting_df.replace({'HOF': {'N':0, 'Y': 1}}, inplace=True)
batting_df.head()

,2B,3B,BA,BB,HBP,HOF,HR,IBB,OBP,OPS,...,SF,SLG,firstName,games,hits,lastName,player_id,runs,seasons,WAR
0,0.01,0.01,0.002500,0.01,0.01,0,0.01,0.01,0.007444,0.024944,...,0.01,0.017500,Francis Patterson,1.0,0.01,Abercrombie,abercda01,0.01,1.0,-0.07
1,35.00,12.02,0.277011,19.01,0.07,0,1.06,0.07,0.288029,0.615584,...,0.07,0.327555,Robert Edward,274.0,341.00,Addy,addybo01,227.00,7.0,2.89
2,19.00,8.02,0.254054,5.03,0.06,0,1.05,0.06,0.259129,0.564791,...,0.06,0.305662,Arthur Algernon,168.0,188.00,Allison,allisar01,106.00,6.0,0.20
3,581.00,142.00,0.332587,983.00,32.16,0,97.10,0.27,0.392578,0.837679,...,0.27,0.445101,Adrian Constantine,2523.0,3418.00,Anson,ansonca01,1996.00,27.0,93.92
4,2.00,1.00,0.224490,0.01,0.01,0,0.01,0.01,0.224760,0.531495,...,0.01,0.306735,Robert,12.0,11.00,Armstrong,armstbo01,9.00,1.0,0.00


In [3]:
# PRINT ORIGINAL DF SHAPE AND DISPLAY COUNTS OF HOF 
print(batting_df.shape)
batting_df['HOF'].value_counts()

(12715, 21)


0    12035
1      680
Name: HOF, dtype: int64

In [4]:
### THIS IS JUST FOR TESTING THE SPLIT BASED ON PANDAS FILTERS
test = batting_df[(batting_df['BA'] < .400) & (batting_df['SLG'] < .700)& (batting_df['seasons'] > 4)]
a = test['HOF'].value_counts()
split = (a[0] - a[1])/test.shape[0]
print(a)
print(split)

0    4585
1     677
Name: HOF, dtype: int64
0.742683390345876


In [5]:
# CELL USED TO DISPLAY MEAN, MIN, MAX OF A PARTICULAR COLUMN
# SOLELY FOR DATA REVIEW
d = 'WAR'

print("MAX " + d + ": " + str(batting_df[d].max()))
print("MEAN " + d + ": " + str(batting_df[d].mean()))
print("MIN " + d + ": " + str(batting_df[d].min()))

MAX WAR: 162.43
MEAN WAR: 3.6919858434919206
MIN WAR: -13.48


In [6]:
# FILTERS FOR BA AND SLG ARE USED TO REMOVE OUTLIERS
# 5 OR MORE SEASONS DETERMINES A GENERALLY ESTABLISHED HITTER
BAT = batting_df[(batting_df['BA'] < .400) & (batting_df['SLG'] < .700)& (batting_df['seasons'] > 4)]
target = BAT['HOF']
print(BAT.shape)
target_names = ['negative', 'positive']
data =BAT.drop('HOF', axis=1)
feature_names = data.columns
# FINAL EVALUATED VALUES DETERMINED TO BE OF IMPORTANCE FOR OFFENSE STATS
d = data[['BA','WAR','OBP','OPS', 'SLG','seasons']]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(d, target, random_state=42)
d.head()

(5262, 21)


,BA,WAR,OBP,OPS,SLG,seasons
1,0.277011,2.89,0.288029,0.615584,0.327555,7.0
2,0.254054,0.20,0.259129,0.564791,0.305662,6.0
3,0.332587,93.92,0.392578,0.837679,0.445101,27.0
9,0.243043,0.80,0.254181,0.584867,0.330686,6.0
20,0.269424,8.47,0.273697,0.593751,0.320054,9.0


In [7]:
# RUN SCORE EVALUATION ON TEST DATA
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)


0.9179331306990881

In [8]:
# PRINT CLASSIFICATION REPORT
from sklearn.metrics import classification_report
predictions = rf.predict(X_test)
print(classification_report(y_test, predictions,
                            target_names=["Positive","Negative"]))

             precision    recall  f1-score   support

   Positive       0.94      0.97      0.95      1146
   Negative       0.74      0.56      0.64       170

avg / total       0.91      0.92      0.91      1316



In [9]:
# RUN PREDICTIONS ON THE PRIMARY ANALYZED DATA
# ASSIGN THE PREDICTIONS TO A COLUMN VALUE IN THE DF
d['predictions'] = rf.predict(d)

/Users/kristopherking/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
# CONCATENATE THE PREDICTED RESULTS ON THE INDEX
# THE PLAYERID CAN NOW BE VALIDATED WITH THE RESULTS
final_batting = pd.concat([BAT, d], axis=1)
final_batting.head()

,2B,3B,BA,BB,HBP,HOF,HR,IBB,OBP,OPS,...,runs,seasons,WAR,BA,WAR,OBP,OPS,SLG,seasons,predictions
1,35.00,12.02,0.277011,19.01,0.07,0,1.06,0.07,0.288029,0.615584,...,227.0,7.0,2.89,0.277011,2.89,0.288029,0.615584,0.327555,7.0,0
2,19.00,8.02,0.254054,5.03,0.06,0,1.05,0.06,0.259129,0.564791,...,106.0,6.0,0.20,0.254054,0.20,0.259129,0.564791,0.305662,6.0,0
3,581.00,142.00,0.332587,983.00,32.16,0,97.10,0.27,0.392578,0.837679,...,1996.0,27.0,93.92,0.332587,93.92,0.392578,0.837679,0.445101,27.0,0
9,15.02,16.02,0.243043,8.01,0.06,0,0.06,0.06,0.254181,0.584867,...,109.0,6.0,0.80,0.243043,0.80,0.254181,0.584867,0.330686,6.0,0
20,78.00,14.03,0.269424,14.03,0.09,0,5.05,0.09,0.273697,0.593751,...,405.0,9.0,8.47,0.269424,8.47,0.273697,0.593751,0.320054,9.0,0


In [14]:
# CONVERT FINAL DF THAT CONTAINS ACTUAL AND PREDICTED HOF VALUES IN TO A CSV RESOURCE
final_batting.to_csv('final_data/batting_dataFinal.csv')